# GPRDR benchmark

I've written this benchmark to reconstruct work done by Duo Li and Alice-Agnes Gabriel from LMU. Ensure you've set the Python path correct

export PYTHONPATH=../../../python

to Peano's Python directory before you invoke the notebook.

In [1]:
import os
import peano4
import exahype2

DaStGen 2 (C) www.peano-framework.org
Peano 4 (C) www.peano-framework.org
ExaHyPE 2 (C) www.peano-framework.org


## Clean-up

Lets ensure that no left-over files stay in there. Lets in particular delete the executable and the output files.

In [2]:
output_files = [ f for f in os.listdir(".") if f.endswith(".peano-patch-file") or f.endswith(".vtu") or f.startswith("output")]
for f in output_files:
  os.remove(f)

## Setup simulation

We create a new simulation. At the moment I can't non-cubic domains. I mean: I haven't tested it. This is something I have to do.

In [3]:
project = exahype2.Project( 
  namespace    = ["examples", "exahype2", "gprdr"], 
  project_name = "gprdr", 
  directory    = "." )

In [4]:
dimensions=2 
project.set_global_simulation_parameters(
  dimensions            = dimensions,
  offset                = [-10000.0,-10000.0], 
  size                  = [20000.0,20000.0],
  end_time              = 8.001,
  first_plot_time_stamp = 0.0, 
  time_in_between_plots = 0.1
)

## Create the solver

I have a whole set of different solvers and will definitely create a few more over the next months. This guy is the most basic/primitive one. Lets ensure it works first. I work with patch-based AMR with patches of the size 5x5 volumes. This is, so far, also a regular grid setup.

In [5]:
mesh_size      = 300
time_step_size = 0.01

unknowns = {    # yeah, I like self-explaining names ;-)
  "rho":1,
  "v":  3,
  "s":  1,
  "J":  3,
  "A":  9,
  "alpha": 1,
  "lambda": 1,
  "mu": 1,
  "xi": 1,
  "xi1": 1,
  "sliding": 1,
  "sliding_xi": 1,
  "slip": 3
}
parameters = {
  "TPV3": 3
}

number_of_unknowns = 0
number_of_auxiliary_variables = 0
for i in unknowns:
  number_of_unknowns += unknowns[i]
for i in parameters:
  number_of_auxiliary_variables += parameters[i]

my_solver = exahype2.solvers.fv.GenericRusanovFixedTimeStepSize(
    name           = "GPRDR", 
    patch_size     = 5, 
    unknowns       = number_of_unknowns,
    auxiliary_variables = number_of_auxiliary_variables,
    time_step_size = time_step_size, 
    min_h          = mesh_size,
    max_h          = mesh_size,
    flux=exahype2.solvers.fv.PDETerms.User_Defined_Implementation,
    ncp=exahype2.solvers.fv.PDETerms.User_Defined_Implementation)
    
project.add_solver( my_solver )    

## Configure the build

The idea is that the ExaHyPE2 front-end generates a Peano Python project. We can then ask this Peano project to generate all C++ code that we need. As we don't want to type in all the different configuration parameters (which compiler is used, which environment variables are to be used on this machine, ...) we first tell the ExaHyPE project to parse the setups we've selected through the initial ./configure call. Ah, and we also commit to a build flavour. 

In [6]:
build_mode = peano4.output.CompileMode.Asserts
#build_mode = peano4.output.CompileMode.Release

project.set_Peano4_installation("../../..", build_mode)

peano4_project = project.generate_Peano4_project()

parse configure outcome ../../../src/Makefile to extract compile settings
used C++ compiler is g++
used Fortran compiler is gfortran


Next, we add all the Fortran routines that we need in our project. Peano 4/ExaHyPE 2 will automatically add it to the build environment later on.

OK, time to generate the actual C++ stuff. After that, we can compile. We can either use the Python front-end to compile (there's a build() command) or directly call make. Once Peano 4 has written the makefile and the C++ code, the code is stand-alone, i.e. doesn't need Python anymore. So a plain make does the job.

In [7]:
peano4_project.generate()

generate all code ...
some artefacts have already been added to repository ... assume this is intentional (by higher abstraction layer, e.g.)
generated gprdr-main.cpp
write ./Makefile
write ././AbstractGPRDR.h (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRusanovFixedTimeStepSizeAbstract.template.h)
write ././AbstractGPRDR.cpp (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRusanovFixedTimeStepSizeAbstract.template.cpp)
write ././GPRDR.h (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRusanovFixedTimeStepSize.template.h)
write ././GPRDR.cpp (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRusanovFixedTimeStepSize.template.cpp)
write ./observers/SolverRepository.h (generated from template /home/tobias/git/Peano/python/exahype2/ProjectSolverRepository.template.h)
write ./observers/SolverRepository.cpp (generated from template /home/tobias/git/Peano/pyth

In [8]:
# parallel_builds = 8
# peano4_project.build( make_clean_first = True, number_of_parallel_builds = parallel_builds )
!make


g++ -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o AbstractGPRDR.o AbstractGPRDR.cpp
g++ -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o GPRDR.o GPRDR.cpp
GPRDR.cpp: In member function ‘virtual double examples::exahype2::gprdr::GPRDR::maxEigenvalue(double*, const tarch::la::Vector<2, double>&, const tarch::la::Vector<2, double>&, double, int)’:
GPRDR.cpp:42:1: warning: no return statement in function returning non-void [-Wreturn-type]
   42 | }
      | ^
g++ -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o observers/SolverRepository.o observers/Solve

g++ -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o observers/TimeStep2peano4_toolbox_blockstructured_BackupPatchOverlap4.o observers/TimeStep2peano4_toolbox_blockstructured_BackupPatchOverlap4.cpp
g++ -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o observers/TimeStep2exahype2_grid_SetLabels5.o observers/TimeStep2exahype2_grid_SetLabels5.cpp
g++ -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=2 -I. -I../../../src -c -o observers/TimeStep.o observers/TimeStep.cpp
g++ -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/inte

## How to modify the actual code

This is ExaHyPE 2, but on the user side it is almost 1:1 the same interface as the original ExaHyPE. All the knowledge can be found in the files GPRDR.h and GPRDR.cpp. These files have been generated by the Python notebook, as you have named the solver GPRPR above.